In [ ]:
import pandas as pd
from Bio import SeqIO
import re
import os

# --- CẤU HÌNH ---
FASTA_PATH = "Homo_sapiens.GRCh38.pep.all.fa"
CSV_PATH = "variant_annotated_official_clean.csv"
OUTPUT_PATH = "variant_protein_sequence_101aa.csv"

WINDOW = 50 
TARGET_LEN = 101 # 50 + 1 (đột biến) + 50
PAD_CHAR = 'X'

In [ ]:
# --- HÀM TIỆN ÍCH ---

def load_ensembl_protein_fasta(path):
    """
    Kế thừa logic load genome: Map từ Transcript ID (ENST) sang Protein Sequence.
    """
    mapping = {}
    print(f"🧬 Đang nạp FASTA từ {path}...")
    for record in SeqIO.parse(path, "fasta"):
        # Trích xuất ENST từ header: ... transcript:ENST00000641515.2 ...
        match = re.search(r'transcript:(ENST\d+)', record.description)
        if match:
            enst_id = match.group(1)
            mapping[enst_id] = str(record.seq).upper()
    print(f"✅ Đã nạp {len(mapping)} mã transcript.")
    return mapping

def normalize_centered_protein(seq, center_idx, target_len, pad_char='X'):
    """
    Kế thừa hoàn toàn logic 'Symmetric Crop/Pad' từ notebook DNA của bạn.
    """
    half = target_len // 2
    start = center_idx - half
    end = center_idx + half + 1
    
    pad_left = max(0, -start)
    pad_right = max(0, end - len(seq))
    
    crop_left = max(0, start)
    crop_right = min(len(seq), end)
    
    final_seq = (pad_char * pad_left) + seq[crop_left:crop_right] + (pad_char * pad_right)
    return final_seq[:target_len] # Đảm bảo luôn đủ 101

In [ ]:
# --- LOGIC XỬ LÝ CHÍNH ---

# 1. Load FASTA
protein_dict = load_ensembl_protein_fasta(FASTA_PATH)

# 2. Đọc CSV theo chunk (cho 3 triệu dòng)
print(f"📊 Đang xử lý file CSV: {CSV_PATH}")
chunk_size = 100000
first_chunk = True

for chunk in pd.read_csv(CSV_PATH, chunksize=chunk_size):
    results = []
    
    for idx, row in chunk.iterrows():
        try:
            # Trích xuất Transcript ID (Feature)
            enst_id = str(row['Feature']).split('.')[0]
            if enst_id not in protein_dict:
                results.append((None, None))
                continue
            
            full_ref_protein = protein_dict[enst_id]
            
            # Xử lý vị trí (Lấy số đầu tiên nếu là dải 226-228)
            pos_str = str(row['Protein_position']).split('-')[0]
            if pos_str == '-':
                results.append((None, None))
                continue
            pos_1based = int(pos_str)
            pos_0based = pos_1based - 1
            
            # Xử lý Amino Acid (Ref/Alt)
            aa_change = str(row['Amino_acids'])
            if '/' not in aa_change:
                results.append((None, None))
                continue
            ref_aa_part, alt_aa_part = aa_change.split('/')
            
            # --- XÁC MINH (Kế thừa verify_ref_seq_center) ---
            actual_ref_aa = full_ref_protein[pos_0based : pos_0based + len(ref_aa_part)]
            if actual_ref_aa != ref_aa_part:
                # Nếu không khớp tuyệt đối, đánh dấu lỗi hoặc bỏ qua
                results.append((None, None))
                continue

            # --- TẠO CHUỖI ALT ---
            # Thay thế đoạn ref bằng alt
            alt_full_protein = (full_ref_protein[:pos_0based] + 
                                alt_aa_part + 
                                full_ref_protein[pos_0based + len(ref_aa_part):])
            
            # XỬ LÝ NONSENSE/STOP: Nếu gặp * hoặc X trong phần Alt
            # Biến tất cả phía sau điểm dừng thành PAD_CHAR 'X'
            if '*' in alt_full_protein or 'X' in alt_aa_part:
                stop_idx = alt_full_protein.find('*')
                if stop_idx == -1: stop_idx = alt_full_protein.find('X', pos_0based)
                # Giữ lại ký tự dừng, còn lại phía sau là X
                alt_full_protein = alt_full_protein[:stop_idx+1] + (PAD_CHAR * (len(full_ref_protein)))

            # --- CẮT CỬA SỔ 101 AA ---
            ref_101 = normalize_centered_protein(full_ref_protein, pos_0based, TARGET_LEN, PAD_CHAR)
            alt_101 = normalize_centered_protein(alt_full_protein, pos_0based, TARGET_LEN, PAD_CHAR)
            
            results.append((ref_101, alt_101))
            
        except:
            results.append((None, None))

    # Gán kết quả vào chunk
    chunk['prot_ref_seq'], chunk['prot_alt_seq'] = zip(*results)
    
    # Lưu xuống file (Append mode)
    chunk.dropna(subset=['prot_ref_seq']).to_csv(
        OUTPUT_PATH, 
        mode='a' if not first_chunk else 'w', 
        index=False, 
        header=first_chunk
    )
    first_chunk = False
    print(f" > Đã lưu xong một cụm dữ liệu...")

print(f"🎉 Hoàn thành! File kết quả: {OUTPUT_PATH}")